In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename
from PIL import Image

# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),  # ResNet-18 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
dataset = ImageFolder('/kaggle/input/satellite-image-classification/data/', transform=transform)

# Split the dataset into training and testing sets
test_size = 0.2
num_dataset = len(dataset)
num_test = int(num_dataset * test_size)
num_train = num_dataset - num_test

train_set, test_set = random_split(dataset, [num_train, num_test])

# Create data loaders
batch_size = 16
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Define and fine-tune the ResNet-18 model
class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Create an instance of the model
model = CustomResNet(num_classes=len(dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

# Define a Flask web application
app = Flask(__name__)

# Define a route for the image classification page
@app.route('/')
def index():
    return render_template('index.html')

# Define a route for image classification
@app.route('/classify', methods=['POST'])
def classify():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join('uploads', filename)
        file.save(file_path)

        # Open and preprocess the uploaded image
        img = Image.open(file_path).convert('RGB')
        img = transform(img).unsqueeze(0)  # Add batch dimension

        # Make a prediction
        model.eval()
        with torch.no_grad():
            output = model(img)

        predicted_class = dataset.classes[torch.argmax(output, dim=1).item()]

        return jsonify({'class': predicted_class, 'confidence': str(torch.max(output).item())})


if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'torch'

In [2]:
!pip install torch


     ------------------------------------ 172.4/172.4 MB 158.8 kB/s eta 0:00:00
